In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from tqdm.keras import TqdmCallback

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
data_dir = "../data/cropped"

In [3]:

def create_dataframe(dir):
    image_paths = []
    card_type = []

    for cardtype in os.listdir(dir):
        cardtype_path = os.path.join(dir, cardtype)

        if os.path.isdir(cardtype_path):
            for filename in os.listdir(cardtype_path):
                if filename.endswith('jpg'):
                    image_paths.append(os.path.join(dir, cardtype, filename))
                    # splitting by shading
                    card_type.append(cardtype[0:1 ])
    return image_paths, card_type

In [4]:
data = pd.DataFrame()
data['images'], data['card type'] = create_dataframe(data_dir)

In [5]:
def reshape_images(images):
    reshaped = []
    for image in tqdm(images):
        img = load_img(image)
        img = np.array(img)
        reshaped.append(img)
    reshaped = np.array(reshaped)
    reshaped = reshaped.reshape(len(reshaped), 200, 150, 3)

    return reshaped

In [6]:
data_reshaped = reshape_images(data['images'])

x = data_reshaped / 255.0

  0%|          | 0/2307 [00:00<?, ?it/s]

In [7]:
le = LabelEncoder()

le.fit(data['card type'])


y = le.transform(data['card type'])
y = to_categorical(y, num_classes=3)

In [8]:
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (2307, 200, 150, 3)
Shape of y: (2307, 3)


In [9]:
model = Sequential()

model.add(Input(shape=(200, 150, 3)))
model.add(Conv2D(32, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

In [10]:
model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(3, activation= "softmax"))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
checkpoint = ModelCheckpoint("../models/color.keras", monitor = "val_accuracy", verbose = 1, save_best_only = True, mode = 'max')

early_stopping = EarlyStopping(monitor = "val_loss", min_delta = 0.001, patience = 5, verbose = 1, restore_best_weights = True)

reduced_learning_rate = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3, verbose = 1, min_delta = 0.0001)


callbacks_list = [checkpoint, early_stopping, reduced_learning_rate]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [13]:
datagen = ImageDataGenerator(
    brightness_range=[0.8, 1.2]
)

datagen.fit(x_train)

In [14]:
batch_size = 32
epochs = 50

In [15]:
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) // batch_size,
    validation_data=(x_test, y_test),
    epochs=epochs,
    callbacks=callbacks_list
)

Epoch 1/50


2024-07-30 15:15:34.601510: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


57/57 [==============================] - ETA: 0s - loss: 67.7780 - accuracy: 0.3486
Epoch 1: val_accuracy improved from -inf to 0.36364, saving model to ../models/color.keras
57/57 [==============================] - 26s 444ms/step - loss: 67.7780 - accuracy: 0.3486 - val_loss: 1.0980 - val_accuracy: 0.3636 - lr: 0.0010
Epoch 2/50
57/57 [==============================] - ETA: 0s - loss: 0.9809 - accuracy: 0.4721
Epoch 2: val_accuracy did not improve from 0.36364
57/57 [==============================] - 25s 437ms/step - loss: 0.9809 - accuracy: 0.4721 - val_loss: 1.0972 - val_accuracy: 0.3636 - lr: 0.0010
Epoch 3/50
57/57 [==============================] - ETA: 0s - loss: 0.7411 - accuracy: 0.6271
Epoch 3: val_accuracy did not improve from 0.36364
57/57 [==============================] - 25s 441ms/step - loss: 0.7411 - accuracy: 0.6271 - val_loss: 1.0960 - val_accuracy: 0.3636 - lr: 0.0010
Epoch 4/50
57/57 [==============================] - ETA: 0s - loss: 0.5809 - accuracy: 0.7286
Epoch

KeyboardInterrupt: 

In [ ]:
# Checking the train and test loss and accuracy values from the neural network above.
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
test_accuracy = history.history['val_accuracy']

# Plotting a line chart to visualize the loss and accuracy values by epochs.
fig, ax = plt.subplots(ncols=2, figsize=(15,7))
ax = ax.ravel()
ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)
ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)
fig.suptitle(x=0.5, y=0.92, t="Lineplots showing loss and accuracy of CNN model by epochs", fontsize=16)

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('../models/color.keras')

In [ ]:
def prep(image):
    img = load_img(image)
    feature = np.array(img)
    feature = feature.reshape(1, 200, 150, 3)
    return feature / 255.0

In [ ]:
def predict_number(imagepath):
    image = prep(imagepath)
    pred = model.predict(image).argmax()

    if pred == 0:
        return "Red", 0
    if pred == 1: 
        return "Green", 1
    else:
        return "Purple", 2


In [ ]:
imagepath = '../cropped/3111/10.jpg'

pred = predict_number(imagepath)
print("prediction: ", pred[0])

loaded_image = cv2.imread(imagepath)
cv2.imshow('image ', loaded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()